In [74]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os
import numpy as np 
import scipy


In [75]:
num_classes=6
img_rows,img_cols=48,48
batch_size=32


In [76]:
train_data_dir='fer2013/train'
validation_data_dir='fer2013/validation'


In [77]:
train_datagen = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					width_shift_range=0.4,
					height_shift_range=0.4,
					horizontal_flip=True,
					fill_mode='nearest')

In [78]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [79]:

train_generator = train_datagen.flow_from_directory(
					train_data_dir,
					color_mode='grayscale',
					target_size=(img_rows,img_cols),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)

Found 28353 images belonging to 6 classes.


In [80]:
validation_generator = validation_datagen.flow_from_directory(
							validation_data_dir,
							color_mode='grayscale',
							target_size=(img_rows,img_cols),
							batch_size=batch_size,
							class_mode='categorical',
							shuffle=True)

Found 3534 images belonging to 6 classes.


In [81]:

model = Sequential()


In [82]:
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [83]:
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


In [84]:
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [85]:
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


In [86]:

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [87]:
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


In [88]:

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))


In [89]:

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
activation_22 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_23 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 24, 24, 32)       

In [90]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [91]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

In [92]:
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

In [93]:

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

In [94]:
callbacks = [earlystop,checkpoint,reduce_lr]

In [95]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])


In [96]:
nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25


In [97]:
history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

Epoch 1/25


2021-08-20 16:09:31.949180: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 26542080 exceeds 10% of free system memory.
2021-08-20 16:09:31.950774: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 26542080 exceeds 10% of free system memory.
2021-08-20 16:09:32.065656: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 29196288 exceeds 10% of free system memory.


  1/755 [..............................] - ETA: 29:53 - loss: 2.6354 - accuracy: 0.2188

2021-08-20 16:09:32.460136: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 26542080 exceeds 10% of free system memory.
2021-08-20 16:09:32.460643: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 26542080 exceeds 10% of free system memory.


755/755 [==============================] - 370s 488ms/step - loss: 2.0604 - accuracy: 0.2016 - val_loss: 1.7369 - val_accuracy: 0.2527

Epoch 00001: val_loss improved from inf to 1.73695, saving model to EmotionDetectionModel.h5
Epoch 2/25
755/755 [==============================] - 370s 487ms/step - loss: 1.7653 - accuracy: 0.2414 - val_loss: 1.7272 - val_accuracy: 0.2628

Epoch 00002: val_loss improved from 1.73695 to 1.72717, saving model to EmotionDetectionModel.h5
Epoch 3/25
755/755 [==============================] - 243s 322ms/step - loss: 1.7381 - accuracy: 0.2581 - val_loss: 1.7467 - val_accuracy: 0.2621

Epoch 00003: val_loss did not improve from 1.72717
Epoch 4/25
755/755 [==============================] - 234s 310ms/step - loss: 1.7310 - accuracy: 0.2582 - val_loss: 1.7110 - val_accuracy: 0.2782

Epoch 00004: val_loss improved from 1.72717 to 1.71101, saving model to EmotionDetectionModel.h5
Epoch 5/25
755/755 [==============================] - 235s 311ms/step - loss: 1.7114 